In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import numpy as np


import numpy as np
from bertopic import BERTopic
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm



In [26]:
# import pandas as pd

# Read the first 5000 rows from the CSV file
# df = pd.read_csv('', nrows=10000)
df = pd.read_csv('Ola Customer Reviews.csv', nrows=50000)

# Load the dataset
# df = pd.read_csv('amazon_uk_shoes_products_dataset_2021_12.csv')

# Display the first few rows of the dataframe
df.head()



,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,laguage_code,country_code
0,Google Play,fbc7ffc9-5a89-446e-87fd-d69bf4a7f984,Puipuii Ralte,NaN,"The map in Ola is so messed up, i have to pay ...",1,0,2023-08-10 16:40:50,NaN,NaN,6.3.2,en,in
1,Google Play,5a0051fb-220a-45b2-ba94-a15a2949218f,Deepak Kumar,NaN,Deepak Kumar.... 🙏🙏🙏🙏🙏],5,0,2023-08-10 16:36:14,NaN,NaN,NaN,en,in
2,Google Play,71ebf933-b734-474d-bb65-a18c90906ed2,Ahamed Azarudeen,NaN,Such aa irresponsible app more then I waiting ...,1,0,2023-08-10 16:29:31,NaN,NaN,6.3.1,en,in
3,Google Play,e1cc0010-60b3-4126-99c2-e8549088566a,Rahil Syed,NaN,Worst,1,0,2023-08-10 15:52:06,NaN,NaN,5.0.4,en,in
4,Google Play,77cf1be1-b428-4493-ae25-e0f288f79b8f,vin 007,NaN,Too much expensive .. try UBer... They are pro...,1,0,2023-08-10 15:51:10,NaN,NaN,NaN,en,in


In [10]:

# import re
# # Load the dataset
# # df = pd.read_csv('your_dataset.csv')

# # Check the data type of the 'review_date' column
# print(df['review_date'].dtype)

# # Display the first few entries in the 'review_date' column
# print(df['review_date'].head())

# # Fill NaN values with an empty string to avoid issues during regex extraction
# df['review_date'] = df['review_date'].fillna('')

# # Convert all values in 'review_date' to strings to ensure consistency
# df['review_date'] = df['review_date'].astype(str)

# # Extract country using regular expression and create a new column
# pattern = r"Reviewed in ([\w\s]+) on"
# df.loc[:, 'country'] = df['review_date'].apply(lambda x: re.search(pattern, x).group(1))

# # Count the occurrences of each unique value
# counts = df['country'].value_counts()

# # Display the counts
# print(counts)

# # Check for rows with NaN values in 'country'
# nan_rows = df[df['country'].isna()]
# print("Rows with NaN values in 'country':\n", nan_rows)

# # Display the first few rows to verify
# print(df.head())

# # Filtering out English only dataset
# df = df[df['review_date'].str.contains('United States', na=False)]
# print("Number of reviews from the United States:", len(df))


In [27]:
# 
# Pre-processing 1: Preprocessing function using TensorFlow
# 
# 
def preprocess_text(text):
    # Convert to lowercase
    text = tf.strings.lower(text)
    # Remove HTML tags
    text = tf.strings.regex_replace(text, r'<.*?>', '')
    # Remove non-alphabet characters
    text = tf.strings.regex_replace(text, r'[^a-zA-Z\s]', '')
    # Strip leading and trailing whitespaces
    text = tf.strings.strip(text)
    return text.numpy().decode('utf-8')

# Remove rows with NaN values in the review_text column
df = df.dropna(subset=['review_description'])

# Select rows where the length of review_description is greater than or equal to 20 characters
df = df[df['review_description'].str.len() >= 20]

# Check the number of rows in the DataFrame after removing NaN values
num_rows = len(df)
print(f"Number of rows in the DataFrame after removing NaN values: {num_rows}")

# Apply preprocessing
df['cleaned_review'] = df['review_description'].apply(lambda x: preprocess_text(tf.convert_to_tensor(x)))


# Display the first few rows of the dataframe with cleaned reviews
df[['review_description', 'cleaned_review']].head()


# Check for NaN values in the review_text column
nan_count = df['cleaned_review'].isna().sum()
print(f"Number of NaN values in review_text column: {nan_count}")

Number of rows in the DataFrame after removing NaN values: 38388
Number of NaN values in review_text column: 0


In [5]:
# 
# Pre-processing 2: Train a count vecotrizer to identify stopwords
# 
# 

from sklearn.feature_extraction.text import CountVectorizer

# Vectorize the text using CountVectorizer with n-grams
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(df['cleaned_review'])

# Display the top n-grams
feature_names = vectorizer.get_feature_names_out()
print(len(feature_names))

# Get the feature names (keys)
feature_names = vectorizer.get_feature_names_out()

# Sum the counts of each feature across all documents
feature_counts = X.toarray().sum(axis=0)

# Create a DataFrame to display feature names and their counts
feature_df = pd.DataFrame({'feature': feature_names, 'count': feature_counts})
print(feature_df)

feature_df.to_csv('count_vector.csv', index = False)

3483
     feature  count
0         aa     19
1      aadhe      3
2        aaj     12
3       aane      2
4        aap     80
...      ...    ...
3478   youve      5
3479      yr      3
3480    zada      4
3481    zero     52
3482   zyada      4

[3483 rows x 2 columns]


In [28]:
import nltk
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Download stop words
nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))

# Sample Hinglish stopwords list (you can expand this list)
hinglish_stopwords = {
    'hai', 'ho', 'main', 'mein', 'tum', 'kya', 'kyun', 'ke', 'ko', 'se', 'ka', 'ki', 'par', 
    'ek', 'aur', 'aur', 'hi', 'kar', 'raha', 'rahe', 'rahi', 'tha', 'thi', 'the', 'hai', 
    'hun', 'hona', 'diya', 'gaya', 'ke', 'ko', 'ne', 'se', 'me', 'ka', 'ki', 'ho', 'kyun',
    'nahi', 'nhi', 'bhi', 'koi', 'kiya', 'ye', 'aap', 'na', 'hota', 'jab', 'tab', 'bahut',
    'dete', 'bar', 'karte', 'kabhi', 'tabhi', 'karo', 'kam', 'kr', 'pe', 'phir', 'sabse',
    'best', 'good', 'worst', 'ola', 'nice', 'wrost', 'ghatiya', 'bekar', 'bakwas', 'fir', 'baad',
    'hain', 'mujhe', 'le', 'jana', 'liye', 'hu', 'de', 'karta', 'karne', 'wale', 'karna', 'ni', 'mera',
    'kuch', 'ha', 'tak', 'aapne', 'main', 'apne', 'jo', 'mene', 'mein', 'pr', 'mere',
    "ola", "worst", "dont", "bad", "use", "rs", "just", "times", "pathetic",
    "hai", "good", "way", "didnt", "better", "want", "using", "got", "like",
    "im", "se", "wont", "ke", "nhi", "aap", "kar", "does", "bhi", "nahi", "ka",
    "hi", "ki", "unki"
}





# Get English stopwords from NLTK
english_stopwords = set(stopwords.words('english'))

# Combine Hinglish and English stopwords
combined_stopwords = hinglish_stopwords.union(english_stopwords)

print(combined_stopwords)



# Function to remove stop words and lemmatize documents
def remove_stopwords(doc):
    tokens = doc.split()
    tokens = [word for word in tokens if word.lower() not in combined_stopwords]
    tokens = [lemmatizer.lemmatize(word, pos=wordnet.VERB) for word in tokens]
    return ' '.join(tokens)

# Remove stop words from the reviews
df['cleaned_review'] = df['cleaned_review'].apply(remove_stopwords)
# Select rows where the length of review_description is greater than or equal to 100 characters
df = df[df['cleaned_review'].str.len() >= 20]
# # Add a new column 'review_length' that contains the length of each review
# # df['review_length'] = df['cleaned_review'].str.len()
# print(len(df))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rahulpoonia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'over', 'was', 'its', 'thi', 'their', 'are', 'own', 'liye', 'your', 'again', 'kyun', 'then', "hadn't", 'mujhe', 'by', 'par', 'hadn', 'where', 'ke', 'rs', 'times', 'tak', 'want', 'did', 'am', 'll', 'than', 'under', 'apne', "should've", 'mustn', 'won', 'pathetic', 'aap', 'fir', 'itself', 'up', 'way', 'y', "you'd", 'above', 'had', 'best', 'like', 'through', 'do', 've', 'aapne', 'he', 'so', 'you', 'himself', 'from', 'having', "haven't", 'too', 'yourself', 'kya', 'pr', 'diya', 'rahe', 'some', 'wrost', 'aren', 'kuch', 'nor', 'to', 'or', 'hasn', 'hona', 'use', "you've", 'phir', 'for', 'yourselves', 'nahi', 'they', "she's", "aren't", 'hi', 'between', 'whom', 'ko', 'karne', 'why', 'once', 'will', 'ni', 'few', 'pe', 'hota', 'before', 'koi', 'into', "wasn't", 'bad', 'didnt', 'dete', 'the', "hasn't", 'after', 'being', 'ye', 'ourselves', 'de', 'karta', 'any', 'my', 'has', 'o', 'more', "didn't", 'when', 'his', 'hu', 'tabhi', 'same', 'and', 'our', 'on', 'does', 'very', 'until', 'should', 'against', 

In [29]:
import umap
# from bertopic import BERTopic
import hdbscan
from sklearn.cluster import KMeans

# Create a HDBSCAN model with adjusted parameters
# hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=2, metric='euclidean')
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=20, 
                                min_samples=5, metric='euclidean', 
                                cluster_selection_method='eom', prediction_data=True)


# Customize UMAP parameters
umap_model = umap.UMAP(n_neighbors=5, n_components=5,
                       min_dist=0.0, metric='cosine', low_memory=False)

# Initialize KMeans with a predefined number of clusters
num_clusters = 32
kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)




In [11]:
# # Anchors designed to nudge the model towards measuring specific genres
# # anchors = [
# #     ["comfortable", "fit"],
# #     ["quite", "sale"]
# # ]

# # seed_topics = {
# #     0: ["great", "love", "recommend", "excellent"],
# #     1: ["interface", "user-friendly", "slow", "improvement"],
# #     2: ["worst", "crashes", "bugs", "issues"]
# # }

# import numpy as np
# print(np.__version__)



In [9]:
# import pandas as pd
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic
# import numpy as np

# # Example DataFrame
# df = pd.DataFrame({
#     'cleaned_review': [
#         "This is a great app. I love using it every day.",
#         "The interface is not user-friendly and it's slow.",
#         "Excellent features and performance. Highly recommended!",
#         "Needs improvement in terms of speed and usability.",
#         "Worst app ever. Crashes frequently and too many bugs."
#     ]
# })

# # Extract the cleaned reviews as a list
# documents = df['cleaned_review'].tolist()

# # Define seed topics as a list of lists
# seed_topics = [
#     ["great", "love", "recommend", "excellent"],  # Topic 0
#     ["interface", "user-friendly", "slow", "improvement"],  # Topic 1
#     #["worst", "crashes", "bugs", "issues"]  # Topic 2
# ]

# # Load a pre-trained sentence embedding model
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# # Compute embeddings for all documents
# embeddings = embedding_model.encode(documents, show_progress_bar=True)

# # Verify the shape of the document embeddings
# print(f"Document Embeddings Shape: {embeddings.shape}")

# # Compute embeddings for the seed topics
# seed_topic_embeddings = [embedding_model.encode(" ".join(seed_topic)) for seed_topic in seed_topics]

# # Convert list of seed topic embeddings to a numpy array and verify shape
# seed_topic_embeddings = np.array(seed_topic_embeddings)
# print(f"Seed Topic Embeddings Shape: {seed_topic_embeddings.shape}")




In [10]:
# # Initialize BERTopic with the custom embedding model and seed topics
# topic_model = BERTopic(embedding_model=embedding_model, seed_topic_list=seed_topics)
# # topic_model = BERTopic(seed_topic_list=seed_topic_embeddings)

# # Fit the model to your documents using precomputed embeddings
# topics, probabilities = topic_model.fit_transform(documents = df['cleaned_review'])

# # Display the most frequent topics
# print(topic_model.get_topic_freq())

# # Get the words and their c-TF-IDF scores for a specific topic
# topic_number = 0
# print(topic_model.get_topic(topic_number))

# # Visualize the topics
# topic_model.visualize_topics()

In [30]:
## Fit BERTopic model


# topic_model = BERTopic(calculate_probabilities=True)
# topic_model = BERTopic(calculate_probabilities=True, min_topic_size = 5)
# topic_model = BERTopic(calculate_probabilities=True, language = "multilingual")
# topic_model = BERTopic(calculate_probabilities=True, umap_model=umap_model)
# topic_model = BERTopic(seed_topic_list = seed_topics)
topic_model = BERTopic(calculate_probabilities=True, umap_model=umap_model, hdbscan_model = hdbscan_model)
# topic_model = BERTopic(nr_topics=20)
# topic_model = BERTopic(min_topic_size = 30)

# topic_model = BERTopic(min_topic_size=4, n_gram_range=(1, 4), calculate_probabilities=True, similarity_threshold=0.0)
# topics, probs = topic_model.fit_transform(df['cleaned_review'], embeddings=document_embeddings)
topics, probs = topic_model.fit_transform(df['cleaned_review'])

# print(len(topics))
print(isinstance(topics, str))

# print(topic_model.topics_)

# Add topics to the original DataFrame
# df['topic'] = pd.DataFrame(topic_model.topics_, columns=['Topic'])

df['topic'] = pd.DataFrame(topic_model.topics_, columns=['Topic'])

# print(df['topic'])

# Get topic info
topic_info = topic_model.get_topic_info()
print(topic_info)

# df['topic'].head






# Extract the topics and their representative words
topics_list = []
for topic in topic_info['Topic']:
    if topic != -1:  # Ignore outliers
        topic_words = topic_model.get_topic(topic)
        topic_words_str = ", ".join([word[0] for word in topic_words])
        topics_list.append({"Topic": topic, "Words": topic_words_str})

# Convert to DataFrame
topics_df = pd.DataFrame(topics_list)

# Save to CSV
topics_df.to_csv("ola_topics.csv", index=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

False
     Topic  Count                                     Name  \
0       -1  14592                -1_ride_money_book_driver   
1        0   1434                    0_paise_krte_krne_mai   
2        1    603                1_postpaid_post_dues_bill   
3        2    553  2_rude_behaviour_unprofessional_drivers   
4        3    327             3_pending_still_payment_cash   
..     ...    ...                                      ...   
313    312     20         312_threaten_spam_whatsapp_legal   
314    313     20    313_chat_article_resolevd_repeatative   
315    314     20             314_km_frad_lalbaug_lessthan   
316    315     20                315_upi_id_validate_paytm   
317    316     20     316_rapido_uber_administration_touch   

                                        Representation  \
0    [ride, money, book, driver, drivers, pay, ask,...   
1    [paise, krte, krne, mai, jyada, sab, toh, rha,...   
2    [postpaid, post, dues, bill, pay, deactivate, ...   
3    [rude, behav

In [31]:
# reduce topics for it to make some sense, the original number of topics are simply too high
topic_model.reduce_topics(df['cleaned_review'], nr_topics='auto')


# Update topic representation by increasing n-gram range and removing english stopwords
topic_model.update_topics(df['cleaned_review'], topic_model.topics_, n_gram_range=(1, 3))

# Get topic info
topic_info = topic_model.get_topic_info()
print(topic_info)


# Extract the topics and their representative words
topics_list = []
for topic in topic_info['Topic']:
    if topic != -1:  # Ignore outliers
        topic_words = topic_model.get_topic(topic)
        topic_words_str = ", ".join([word[0] for word in topic_words])
        topics_list.append({"Topic": topic, "Words": topic_words_str})


# put the words in a column next to topic
topics_list = []
for topic in topic_model.topics_:
    if topic != -1:  # Ignore outliers
        topic_words = topic_model.get_topic(topic)
        topic_words_str = ", ".join([word[0] for word in topic_words])
        topics_list.append({"Topic": topic, "Words": topic_words_str})

    elif topic == -1:
        topic_words_str = "Un-clustered"
        topics_list.append({"Topic": topic, "Words": topic_words_str})



# Create the DataFrame from the list of dictionaries
df_topics = pd.DataFrame(topics_list)


df.to_csv('ola_df.csv',index = False)
df_topics.to_csv('ola_bertopics_words.csv',index = False)





     Topic  Count                                               Name  \
0       -1  14592                            -1_ride_driver_app_book   
1        0  12461                           0_ride_cancel_app_driver   
2        1   1434                           1_paise_driver_book_krte   
3        2    603                2_postpaid_pay_money_money postpaid   
4        3    294               3_charge_twice_charge twice_customer   
..     ...    ...                                                ...   
101    100     20           100_km_show_show km_km reach destination   
102    101     20                   101_safe_women_safety_safe women   
103    102     20  102_technical_glitch_show technical_technical ...   
104    103     20                103_chat_support_option_chat option   
105    104     20                        104_auto_book auto_book_get   

                                        Representation  \
0    [ride, driver, app, book, drivers, cancel, ser...   
1    [ride, cancel,

In [23]:
# # 
# # Pre-processing 3: Use the list of topics identified in the above steps to find a list of anchor words to get topics like you want
# # 
# # 

# seed_topic_list = [["drug", "cancer", "drugs", "doctor"],
#                    ["windows", "drive", "dos", "file"],
#                    ["space", "launch", "orbit", "lunar"]]


# topic_model = BERTopic(seed_topic_list=seed_topic_list)
# topics, probs = topic_model.fit_transform(df['cleaned_review'])

In [35]:
print(df.shape)
print(df_topics.shape)

# Reset the index for both DataFrames
df_reset = df.reset_index(drop=True)
df_topics_reset = df_topics.reset_index(drop=True)

# Concatenate the DataFrames along the columns (axis=1)
df_combined = pd.concat([df_reset, df_topics_reset], axis=1)

# Select the desired columns, for example, 'column1', 'column2', and 'column3'
selected_columns = df_combined[['review_description', 'rating', 'Words']]

# Save the selected columns to a CSV file
selected_columns.to_csv('selected_columns.csv', index=False)

# Verify the shape of the resulting DataFrame
print(df_combined.shape)  # Should print (34875, 17)

(34875, 15)
(34875, 2)
(34875, 17)


In [24]:
from sklearn.decomposition import LatentDirichletAllocation

# Apply LDA to extract aspects
n_topics = 24  # Adjust the number of topics as needed
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda_matrix = lda_model.fit_transform(X)

# Display the top n-grams for each topic
def display_topics(model, feature_names, no_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        topics[topic_idx] = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
    return topics

no_top_words = 10
topics = display_topics(lda_model, vectorizer.get_feature_names_out(), no_top_words)
for topic, words in topics.items():
    print(f"Topic {topic}: {words}")

Topic 0: ['ride', 'driver', 'app', 'drivers', 'cancel', 'cab', 'able', 'destination', 'book', 'just']
Topic 1: ['driver', 'ola', 'post', 'given', 'paid', 'payment', 'ride', 'worst', 'bad', 'app']
Topic 2: ['application', 'app', 'ola', 'worst', 'scam', 'customer', 'nice', 'showing', 'care', 'extra']
Topic 3: ['uber', 'ola', 'better', 'app', 'far', 'driver', 'expensive', 'use', 'km', 'dont']
Topic 4: ['price', 'ride', 'booking', 'ola', 'fare', 'destination', 'shows', 'time', 'worst', 'app']
Topic 5: ['point', 'company', 'drivers', 'app', 'fraud', 'ola', 'doesnt', 'worst', 'pickup', 'pick']
Topic 6: ['ola', 'booked', 'cab', 'driver', 'cabs', 'charged', 'auto', 'ride', 'complaint', 'didnt']
Topic 7: ['good', 'ola', 'app', 'login', 'drivers', 'experience', 'service', 'dont', 'services', 'just']
Topic 8: ['route', 'driver', 'double', 'charge', 'fare', 'app', 'ola', 'charges', 'extra', 'ride']
Topic 9: ['app', 'account', 'blocked', 'reason', 'support', 'customer', 'location', 'high', 'booking